 # ETL de **"users_items"**
 -------------

En este jupyter notebook se desarrolla la extracción, transformación y carga del conjunto del dataset `"user_items"`

-------------
### Tabla de contenido

- 1.- Introducción
- 2.- Importación de las librerías y funciones personalizadas para el ETL
    - 2.1.- Librerías
    - 2.2.- Funciones Personalizadas
- 3.- Desarrollo del proceso de ETL
    - 3.1.- Extracción de datos
        - 3.1.1.- Carga de archivos "user_items.json.gz"
    - 3.2.- Transformación de datos
        - 3.2.1.- Aspectos generales del Dataframe
        - 3.2.2.- Exploración y tratamiendo de datos vacíos
        - 3.2.3.- Eliminación de datos duplicados
        - 3.2.4.- Datos incorrectos o irrelevantes
        - 3.2.5.- Datos categóricos
    - 3.3.- Carga de Datos
        - 3.3.1 Generación del archivo limpio
-------------

## **<span style="color: #d8572a;">1.- Introducción</span>**

Este trabajo ETL tiene como objetivo procesar e integrar un conjunto de datos que contiene información sobre videojuegos de Steam. El conjunto de datos, titulado "user_items", consta de 88310 filas y 5 columnas, luego de un proceso de limpieza exhaustivo para eliminar filas con valores NaN. Los datos extraídos se transformarán y cargarán en un sistema de destino para su posterior análisis y utilización.

## **<span style="color: #d8572a;">2.- Importación de las librerías y funciones personalizadas para el ETL</span>**

### **<span style="color: #f7b538;">2.1.- Librerías</span>**

In [1]:
import pandas as pd
import numpy as np
import ast
import gzip

### **<span style="color: #f7b538;">2.2.- Funciones Personalizadas</span>**

In [2]:
import Func_personalizadas.FPersonalizadas as FP

## **<span style="color: #d8572a;">3.- Desarrollo del proceso de ETL</span>**

### **<span style="color: #f7b538;">3.1.- Extracción de datos</span>**

#### 3.1.1.- Carga de archivos "user_items.json.gz"

In [3]:
ruta_archivo= r"C:\Users\USUARIO\OneDrive\6.- Data Science\1.- Experiencia Soy Henry Bootcamp\Labs\PI 1\ML_DevOps_Steam_Project\Datasets\users_items.json.gz"

In [4]:
# Definimos una lista vacia
rows = []

# Abrimos gzip-compressed JSON file
with gzip.open(ruta_archivo, 'rb') as f:
    #   Iteramos sobre cada línea en el archivo
    for line in f.readlines():
        #  Decodificamos la línea y la evaluamos como un literal de Python
        rows.append(ast.literal_eval(line.decode('utf-8')))
        
# Convertimos la lista de diccionarios en un DataFrame
df_users_items=pd.DataFrame(rows)

### **<span style="color: #f7b538;">**3.2.- Transformación de datos**</span>**

#### 3.2.1.- Aspectos generales del Dataframe

In [5]:
# Dimensiones del dataframe
df_users_items.shape

(88310, 5)

In [6]:
df_users_items.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [7]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [8]:
df_users_items.user_id.value_counts()

user_id
X03-Suits            3
76561198027488037    3
76561198100326818    3
76561198309337430    3
76561198051777058    3
                    ..
8392158              1
76561198056804863    1
SparklezTheTurtle    1
76561198019707497    1
edward_tremethick    1
Name: count, Length: 87626, dtype: int64

### 3.2.2.- Exploración y tratamiendo de datos vacíos

Eliminamos todas las filas que contengan datos vacíos en todos los campos

In [9]:
## Eliminamos las filas vacías
df_users_items = df_users_items.dropna(how='all').reset_index(drop=True)
## Evaluamos dimensiones
df_users_items.shape

(88310, 5)

Revisamos los tipos de datos y los nulos por cada columna

In [10]:
FP.tabla_tipo_datos(df_users_items)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.0,0.0,0
1,items_count,[<class 'int'>],100.0,0.0,0
2,steam_id,[<class 'str'>],100.0,0.0,0
3,user_url,[<class 'str'>],100.0,0.0,0
4,items,[<class 'list'>],100.0,0.0,0


Revisamos a detalles las columnas que tienen estructura tipo lista, para entender su estructura

In [11]:
col_tipo_list = FP.identificar_columnas_con_listas(df_users_items)

In [12]:
for col in col_tipo_list:    
    print(f"Columna {col} \n Tipo de Dato: ",df_users_items[col][5])

Columna items 
 Tipo de Dato:  [{'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 256, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_forever': 167, 'playtime_2weeks': 0}, {'item_id': '320', 'item_name': 'Half-Life 2: Deathmatch', 'playtime_forever': 3674, 'playtime_2weeks': 0}, {'item_id': '4000', 'item_name': "Garry's Mod", 'playtime_forever': 358, 'playtime_2weeks': 0}, {'item_id': '6570', 'item_name': 'Onimusha 3: Demon Siege', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '220', 'item_name': 'Half-Life 2', 'playtime_forever': 1299, 'playtime_2weeks': 0}, {'item_id': '340', 'item_name': 'Half-Life 2: Lost Coast', 'playtime_forever': 29, 'playtime_2weeks': 0}, {'item_id': '360', 'item_name': 'Half-Life Deathmatch: Source', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '380', 'item_name': 'Half-Life 2: Episode One', 'playtime_forever': 170, 'playtime_2weeks': 0}, {'item_id': '400', 'item

In [13]:
# Se observa el tipo de dato que contiene 'review'
df_users_items['items'][5]

[{'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 256,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: Source',
  'playtime_forever': 167,
  'playtime_2weeks': 0},
 {'item_id': '320',
  'item_name': 'Half-Life 2: Deathmatch',
  'playtime_forever': 3674,
  'playtime_2weeks': 0},
 {'item_id': '4000',
  'item_name': "Garry's Mod",
  'playtime_forever': 358,
  'playtime_2weeks': 0},
 {'item_id': '6570',
  'item_name': 'Onimusha 3: Demon Siege',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '220',
  'item_name': 'Half-Life 2',
  'playtime_forever': 1299,
  'playtime_2weeks': 0},
 {'item_id': '340',
  'item_name': 'Half-Life 2: Lost Coast',
  'playtime_forever': 29,
  'playtime_2weeks': 0},
 {'item_id': '360',
  'item_name': 'Half-Life Deathmatch: Source',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '380',
  'item_name': 'Half-Life 2: Episode One',
  'playtime_forever': 170,
  'playtime_2weeks

### Alcance preliminar del diccionario de datos de "User reviews"

**Descripción:**

Este conjunto de datos contiene información sobre videojuegos. Tras la limpieza de filas no se evidencia valores tipo NaN, el conjunto final consta de 88309 filas y 5 columnas. A continuación, se detalla la descripción de cada columna:

**Columnas:**

* **user_id**: contiene un identificador único del usuario.
* **items_count**: contiene un número entero que indica la cantidad de juegos que ha consumido el usuario.
* **steam_id**: es un número único para la plataforma.
* **user_url**: es la url del perfil del usuario
* **items**: contiene una lista de uno o mas diccionarios de los items que consume cada usuario. Cada diccionario tiene las siguientes claves:
    * **item_id**: es el identificados del item, es decir, del juego.
    * **item_name**: es el nombre del contenido que consume, es decir, del juego.
    * **playtime_forever**: es el tiempo acumulado que un usuario jugó a un juego.
    * **playtime_2weeks**: es el tiempo acumulado que un usuario jugó a un juego en las últimas dos semanas.

**Notas:**

* Las columnas `items` contienen listas de diccionarios.

### 3.2.3.- Eliminación de datos duplicados

Se analizan si hay duplicados teniendo en cuenta la columna de `items`, al ser una columna anidada, esta debe ser normalizada primero.

In [14]:
elementos_duplicados = FP.verifica_duplicados_por_columna(df_users_items, 'user_id')
elementos_duplicados

,user_id,items_count,steam_id,user_url,items
11000,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
29193,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
37062,10outof10matee,56,76561198050688208,http://steamcommunity.com/id/10outof10matee,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
37061,10outof10matee,56,76561198050688208,http://steamcommunity.com/id/10outof10matee,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
6167,111222333444555666888,52,76561198082607692,http://steamcommunity.com/id/11122233344455566...,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
...,...,...,...,...,...
4625,youseeitnowgetout,5,76561198087136132,http://steamcommunity.com/id/youseeitnowgetout,"[{'item_id': '230410', 'item_name': 'Warframe'..."
3473,zandado,107,76561198057890701,http://steamcommunity.com/id/zandado,"[{'item_id': '20', 'item_name': 'Team Fortress..."
34176,zandado,107,76561198057890701,http://steamcommunity.com/id/zandado,"[{'item_id': '20', 'item_name': 'Team Fortress..."
12417,zeroblade,306,76561197970272666,http://steamcommunity.com/id/zeroblade,"[{'item_id': '18110', 'item_name': 'Shattered ..."


Identificación de Items Duplicados:

Se han detectado 1357 items duplicados en el conjunto de datos, sin embargo:

* **Primero:** Se revisan si los review dentro de los datos anidados de 'items' la información
* **Segundo:** Corroborar si se encuentra duplicada o si solo se duplica el 'user_id' porque hay mas de un comentario realizado por ese usuario.

Se debe realizar una verificación aparte para la columna 'items', donde se debe normalizar y analizar los elementos del diccionario

### 3.2.4.- Datos incorrectos o irrelevantes

* En una primera instancia, hasta el momento no hay columnas por eliminar.
* La columna items al ser una columna anidada, podría contener información valiosa como también irrelevante, por lo que requiere un análisis más profundo que se abordará en el siguiente punto.

### 3.2.5.- Datos categóricos

#### 3.2.5.1 Columna 'items'

La columna 'items' se presenta anidada, siendo una lista con uno o mas diccionarios como elementos. Se busca generar una columna por cada diccionario para posteriormente hacer un registro por cada diccionario.

In [15]:
# se normaliza la columna 'items'
df_norm_items = pd.json_normalize(rows, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )
df_norm_items

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


Verificamos la cantidad de datos nulos

In [18]:
# Verificamos la cantidad de datos nulos
FP.tabla_tipo_datos(df_norm_items)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,item_id,[<class 'str'>],100.0,0.0,0
1,item_name,[<class 'str'>],100.0,0.0,0
2,playtime_forever,[<class 'int'>],100.0,0.0,0
3,playtime_2weeks,[<class 'int'>],100.0,0.0,0
4,steam_id,[<class 'str'>],100.0,0.0,0
5,items_count,[<class 'int'>],100.0,0.0,0
6,user_id,[<class 'str'>],100.0,0.0,0
7,user_url,[<class 'str'>],100.0,0.0,0


Verificamos la existencia de duplicados

In [19]:
# Devuelve los duplicados
elementos_duplicados = df_norm_items.loc[df_norm_items.duplicated()]
elementos_duplicados

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
164294,20,Team Fortress Classic,5,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164295,50,Half-Life: Opposing Force,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164296,70,Half-Life,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164297,130,Half-Life: Blue Shift,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164298,220,Half-Life 2,198,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
...,...,...,...,...,...,...,...,...
4898223,213670,South Park™: The Stick of Truth™,725,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898224,221910,The Stanley Parable,53,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898225,261030,The Walking Dead: Season Two,253,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898226,273110,Counter-Strike Nexon: Zombies,0,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...


Se observan 59104 duplicados, por lo que se decide borrarlos. **Manteniendo el primer elemento de cada grupo de duplicados**

In [20]:
df_norm_items = df_norm_items.drop_duplicates(keep='first')
df_norm_items

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


En esta transformación, se puede ver que la columna 'playtime_2weeks' presenta redundancia en la información don la columna 'playtime_forever', por lo que se decide eliminar.

In [21]:
# Se eliminan las columnas 'playtime_2weeks'
df_norm_items = df_norm_items.drop(columns=['playtime_2weeks'])
df_norm_items.columns

Index(['item_id', 'item_name', 'playtime_forever', 'steam_id', 'items_count',
       'user_id', 'user_url'],
      dtype='object')

**Se confirma la limpieza exitosa del dataset**

### **<span style="color: #f7b538;">**3.3.- Carga de datos**</span>**

#### 3.3.1 Generación del archivo limpio

Se guarda el dataframe transformado como `user_items_limpio`

In [22]:
nombre_archivo_limpio = 'Datasets\df_user_items_limpio.csv'
df_norm_items.to_csv(nombre_archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {nombre_archivo_limpio}')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_28788\3999126796.py:1: SyntaxWarning: invalid escape sequence '\d'
  nombre_archivo_limpio = 'Datasets\df_user_items_limpio.csv'


Se guardó el archivo Datasets\df_user_items_limpio.csv
